In [1]:
import os
import requests
from csv import writer
import json
import geojson
import geopandas as gpd
import csv
import pandas as pd
import time


In [2]:
# create folder for the features we interested in
parent_dir = "../data/raw/interested_facilities"
os.makedirs(parent_dir, exist_ok=True)

In [3]:
url_list = [
    'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PARK%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=',
    'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27PRSC%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=',
    'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SESC%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=',
    'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27UNIV%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=',
    'https://services6.arcgis.com/GB33F62SbDxJjwEL/ArcGIS/rest/services/Vicmap_Features_of_Interest/FeatureServer/8/query?where=FEATURE_CODE+%3D+%27SHCE%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=standard&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=REGISTRATION_DATE&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token=',
]
name_list = ['park', 'primary_school', 'secondary_school', 'university', 'shopping_mall',]

def fetch_and_write_data(url, file_name):
    response = requests.get(url)
    content = response.json()

    file_path = os.path.join(parent_dir, file_name)
    
    with open(file_path, 'w', newline='') as g:
        thewriter = writer(g)
        thewriter.writerow(['name', 'longitude', 'latitude'])
        
        for feature in content['features']:
            place_name = feature['properties']['PLACE_NAME']
            longitude, latitude = feature['geometry']['coordinates']
            thewriter.writerow([place_name, longitude, latitude])

for url, name in zip(url_list, name_list):
    fetch_and_write_data(url, f'{name}.csv')

In [4]:
with open('../data/raw/hotosm_aus_health_facilities_points_geojson.geojson', 'r', encoding='utf-8') as f:
    data = geojson.load(f)

with open('../data/raw/interested_facilities/hospital.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['name', 'longitude', 'latitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for feature in data['features']:

        name = feature['properties'].get('name')
        longitude = feature['geometry']['coordinates'][0]
        latitude = feature['geometry']['coordinates'][1]

        writer.writerow({'name': name, 'longitude': longitude, 'latitude': latitude})


## Add the postcode and the suburb name to the facilities that we interested in

In [ ]:
#preprocessing the hospital dataset, delete the empty or NaN name raws
hospital_data = pd.read_csv('../data/raw/interested_facilities/hospital.csv')

# Clean rows where 'name' is empty or NaN
hospital_data_cleaned = hospital_data.dropna(subset=['name'])

# Save the cleaned dataset
hospital_data_cleaned.to_csv('../data/raw/interested_facilities/hospital_clean.csv', index=False)

In [16]:
hospital_df = pd.read_csv("../data/raw/interested_facilities/hospital_clean.csv")
park_df = pd.read_csv("../data/raw/interested_facilities/park.csv")
primary_df = pd.read_csv("../data/raw/interested_facilities/primary_school.csv")
secondary_df = pd.read_csv("../data/raw/interested_facilities/secondary_school.csv")
shoppingmall_df = pd.read_csv("../data/raw/interested_facilities/shopping_mall.csv")
university_df = pd.read_csv("../data/raw/interested_facilities/university.csv")

In [17]:
api_key = "" #To prevent exposure or unknown use has been hidden, please contact Xiaowei Guo to get the key

# #because the daily limited API usage, do not run all the facilities at one time
os.makedirs("../data/raw/intermediate_results", exist_ok=True)


# get postcode and suburb name by latitude and longitude
def get_postcode_suburb(latitude, longitude):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            address_components = data['results'][0]['address_components']
            postcode = None
            suburb = None
            for component in address_components:
                if 'postal_code' in component['types']:
                    postcode = component['long_name']
                if 'locality' in component['types']:
                    suburb = component['long_name']
            return postcode, suburb
    return None, None

# Add postcode and suburb columns to each dataset and process in batches
def add_postcode_suburb_in_batches(df, batch_size=500, delay=60):
    df['postcode'] = ''
    df['suburb'] = ''
    
    for start in range(0, len(df), batch_size):
        batch_df = df[start:start+batch_size]
        for index, row in batch_df.iterrows():
            latitude = row['latitude']
            longitude = row['longitude']
            postcode, suburb = get_postcode_suburb(latitude, longitude)
            df.at[index, 'postcode'] = postcode
            df.at[index, 'suburb'] = suburb
        # Save the progress of each batch
        df.to_csv(f"../data/raw/intermediate_results/intermediate_result_{start // batch_size}.csv", index=False)
        print(f"finished batch {start // batch_size + 1}, wait {delay} seconds to continue...")
        time.sleep(delay)  # wait between batches

    return df

# batch each dataset
batch_size = 500  # Process 500 rows per batch
delay_between_batches = 60  # Wait 60s between each batch

hospital_df = add_postcode_suburb_in_batches(hospital_df, batch_size, delay_between_batches)
park_df = add_postcode_suburb_in_batches(park_df, batch_size, delay_between_batches)
primary_df = add_postcode_suburb_in_batches(primary_df, batch_size, delay_between_batches)
secondary_df = add_postcode_suburb_in_batches(secondary_df, batch_size, delay_between_batches)
shoppingmall_df = add_postcode_suburb_in_batches(shoppingmall_df, batch_size, delay_between_batches)
university_df = add_postcode_suburb_in_batches(university_df, batch_size, delay_between_batches)

# save the updated dataset
park_df.to_csv("../data/curated/Facilities/park.csv", index=False)
primary_df.to_csv("../data/curated/Facilities/primary_school.csv", index=False)
secondary_df.to_csv("../data/curated/Facilities/secondary_school.csv", index=False)
shoppingmall_df.to_csv("../data/curated/Facilities/shopping_mall.csv", index=False)
university_df.to_csv("../data/curated/Facilities/university.csv", index=False)
hospital_df.to_csv("../data/curated/Facilities/hospital.csv", index=False)



finished batch 1, wait 60 seconds to continue...
finished batch 2, wait 60 seconds to continue...
finished batch 3, wait 60 seconds to continue...
finished batch 4, wait 60 seconds to continue...
finished batch 5, wait 60 seconds to continue...
finished batch 6, wait 60 seconds to continue...
finished batch 7, wait 60 seconds to continue...
finished batch 8, wait 60 seconds to continue...
finished batch 9, wait 60 seconds to continue...


In [6]:
file_path = '../data/curated/Facilities/hospital.csv'
df = pd.read_csv(file_path)

postcode_column = 'postcode'

df[postcode_column].fillna(0, inplace=True)

df[postcode_column] = df[postcode_column].astype(int)

postcode_count = df.groupby(postcode_column).size().reset_index(name='count')

output_path = '../data/curated/postcode_hospital_count.csv'
postcode_count.to_csv(output_path, index=False)

In [14]:
hospital_file = '../data/curated/postcode_hospital_count.csv'
property_file = '../data/curated/all_Facilities_min_distance.csv'

hospital_df = pd.read_csv(hospital_file)
property_df = pd.read_csv(property_file)

hospital_df.rename(columns={'postcode': 'addressPostcode'}, inplace=True)
hospital_df.rename(columns={'count': 'medical_institution_count'}, inplace=True)

merged_df = pd.merge(property_df, hospital_df, on='addressPostcode', how='left')

merged_df['medical_institution_count'].fillna(0, inplace=True)


output_path = '../data/curated/Final_property_with_features.csv'
merged_df.to_csv(output_path, index=False)

